<a href="https://colab.research.google.com/github/Samin-Sadaf7/QuestionAnswerGenerator/blob/main/TrueFalse_MCQ_Generation_Viva_Question_from_question_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/AI_integration_kernel/MediBee'

/content/gdrive/MyDrive/AI_integration_kernel/MediBee


In [3]:
from huggingface_hub import notebook_login

notebook_login()

#Generate PDF from docx file

In [4]:
!pip install --quiet python-docx reportlab PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 50.6 MB/s eta 0:00:00


In [ ]:
import os
from docx import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
import fitz  # PyMuPDF

# Step 1: Find all .docx files in nested directories
def find_docx_files(root_folder):
    docx_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.endswith('.docx'):
                docx_files.append(os.path.join(dirpath, file))
    return docx_files

# Step 2: Convert .docx to PDF using reportlab
def docx_to_pdf(docx_path, pdf_path):
    doc = Document(docx_path)
    doc_elements = []
    doc_elements.append(canvas.Canvas(pdf_path, pagesize=letter))

    for para in doc.paragraphs:
        text = para.text
        if text.strip():
            doc_elements.append(text)

    for table in doc.tables:
        data = []
        for row in table.rows:
            data.append([cell.text for cell in row.cells])
        doc_elements.append(data)

    c = doc_elements[0]
    width, height = letter
    y = height - 40
    x = 40
    c.setFont("Helvetica", 12)

    for element in doc_elements[1:]:
        if isinstance(element, str):
            text_object = c.beginText(x, y)
            text_object.textLines(element)
            c.drawText(text_object)
            y -= 20
        elif isinstance(element, list):
            table = Table(element)
            table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ]))
            table.wrapOn(c, width - 80, y)
            table.drawOn(c, x, y - (len(element) * 15))
            y -= len(element) * 15 + 20

        if y < 50:
            c.showPage()
            y = height - 40

    c.save()

# Step 3: Merge PDFs into one
def merge_pdfs(pdf_list, output_path):
    merged_pdf = fitz.open()
    for pdf_path in pdf_list:
        pdf_document = fitz.open(pdf_path)
        merged_pdf.insert_pdf(pdf_document)

    merged_pdf.save(output_path)
    merged_pdf.close()

# Main function
def main(root_folder, output_pdf):
    docx_files = find_docx_files(root_folder)
    temp_pdf_files = []

    for docx_file in docx_files:
        temp_pdf_path = f"{os.path.splitext(docx_file)[0]}.pdf"
        docx_to_pdf(docx_file, temp_pdf_path)
        temp_pdf_files.append(temp_pdf_path)

    merge_pdfs(temp_pdf_files, output_pdf)

    # Clean up temporary PDF files
    for temp_pdf in temp_pdf_files:
        os.remove(temp_pdf)

# Example usage
root_folder = '/content/gdrive/MyDrive/AI_integration_kernel/MediBee'  # Update with the path to your root folder
output_pdf = '/content/gdrive/MyDrive/AI_integration_kernel/MediBee/giant.pdf'   # Update with the desired output PDF path
main(root_folder, output_pdf)

#ChromaDB store

In [5]:
import os
from google.colab import userdata
os.environ['Google_API_KEY'] = userdata.get('GeminiKey')

In [6]:
!pip uninstall typer -y
!pip install --force-reinstall spacy weasel

Found existing installation: typer 0.12.3
Uninstalling typer-0.12.3:
  Successfully uninstalled typer-0.12.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 16.1 MB/s eta 0:00

In [1]:
!pip install --force-reinstall --no-deps typer==0.9.0
!pip install fastapi-cli
!pip install langchain-community sentence-transformers pypdf2 pypdf chromadb
!pip install google-generativeai

  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Using cached typer-0.12.3-py3-none-any.whl (47 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Using cached langchain_community-0.2.5-py3-none-any.whl (2.2 MB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

#Imports

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import textwrap

#Create pdf and vds

In [3]:
def split_pdf(file_path, chunk_size, chunk_overlap):
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(pages)
    return docs

In [4]:
def create_VDS(docs, path):
  try:
    # Instantiate HuggingFaceEmbeddings with the specified model
    embedding_function = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory=path)
    print("Number of vectors in the vector store:", vectorstore._collection.count())

  except Exception as e:
    print("An error occurred:", e)
    # If there's an error, print the traceback for debugging
    import traceback
    traceback.print_exc()

#Get Context

In [19]:
def get_context_viva(query, chroma_directory,model_name, k):
  context = ""

  # Instantiate HuggingFaceEmbeddings with the specified model
  embedding_function = HuggingFaceEmbeddings(
      model_name= model_name,
      model_kwargs={'device': 'cpu'}
  )
  Query = "True/False and True False Question for the following topic: " + query + " in medical terminology"
  vector_db = Chroma(persist_directory=chroma_directory, embedding_function=embedding_function)
  search_result = vector_db.similarity_search(query, k)

  for result in search_result:
    context += result.page_content + "\n\n\n"

  return context

In [5]:
def get_context_true_false(query, chroma_directory,model_name, k):

  context = ""

  # Instantiate HuggingFaceEmbeddings with the specified model
  embedding_function = HuggingFaceEmbeddings(
      model_name= model_name,
      model_kwargs={'device': 'cpu'}
  )
  Query = "True/False and True False Question for the following topic: " + query + " in medical terminology"
  vector_db = Chroma(persist_directory=chroma_directory, embedding_function=embedding_function)
  search_result = vector_db.similarity_search(query, k)

  for result in search_result:
    context += result.page_content + "\n\n\n"

  return context

In [6]:
def get_context_sba(query, chroma_directory,model_name, k):

  context = ""

  # Instantiate HuggingFaceEmbeddings with the specified model
  embedding_function = HuggingFaceEmbeddings(
      model_name= model_name,
      model_kwargs={'device': 'cpu'}
  )
  Query = "Multiple Choice Questions or Single Based Answer or MCQ or SBA for the following topic: " + query + " in medical terminology"
  vector_db = Chroma(persist_directory=chroma_directory, embedding_function=embedding_function)
  search_result = vector_db.similarity_search(query, k)

  for result in search_result:
    context += result.page_content + "\n\n\n"

  return context

#Get prompt True False

In [7]:
def get_prompt_TrueFalse(query, context):
  context = context.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""You are assisting a medical student in preparing for an exam. Your task is to generate true false questions based on the given Single Best answer, SQA question and True False Question using\
   medical terminology. You need to avoid making Single Best answer questions even if you get those questions. Ensure your responses are comprehensive and friendly, reflecting a conversational tone. Your only focus is to generate True/False Question and you are given the Question Format.\
   Make sure to generate questions in full sentence. Your task is to generate true false quesions in the given format. \
   The questions should be true of false. The format of question is first mention a topic and follow it with 5-6 sentences, the sentences can be true or false. \
TOPIC: '{query}'
CONTEXT: '{context}'
True/False QUESTIONS Format:
  Topic: T1
  Sentences:
  1. S1
  2. S2
  3. S3
  4. S4
  5. S5
  M1: True
  M2: False
  M3: True
  M4: False
  M5: True
""").format(query=query, context=context)
  return prompt

#Get prompt SBA

In [8]:
def get_prompt_SBA(query, context):
  context = context.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""You are assisting a medical student in preparing for an exam. Your task is to generate multiple choice questions based on the given MCQ, Multiple Choice Question, Single Best answer, SQA question and True False Question using\
   medical terminology. You need to avoid making True False questions even if you get those questions. Ensure your responses are comprehensive and friendly, reflecting a conversational tone. Your only focus is to generate Multiple Choice Question and you are given the Question Format.\
   Make sure to generate questions in full sentence. Your task is to generate Single Based quesions or Multiple Choice Questions in the given format. \
TOPIC: '{query}'
CONTEXT: '{context}'
MCQ QUESTIONS Format:
  Question: Q1
  options: A1, A2, A3, A4
  Answer: A1
  Question: Q2
  options: A1, A2, A3, A4
  Answer: A2
You can create as many mcq questions as you like.
""").format(query=query, context=context)
  return prompt

In [45]:
def get_prompt_Viva(query, context):
  context = context.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""You are assisting a medical student in preparing for an exam. Your task is to generate a viva question based on the given MCQ, Multiple Choice Question, Single Best answer, SQA question and True-False Question using\
   medical terminology.Ensure your responses are comprehensive and friendly, reflecting a conversational tone. Your only focus is to generate Viva Question and you are given the Question Format.\
   Make sure to generate questions in full sentence. Your task is to create the viva question \
   Question should be answerable in 1-2 sentences. You should generate only one question. You should avoid providing answers. \
   The format is
   Topic: T1
   Question: Q1
TOPIC: '{query}'
CONTEXT: '{context}'
""").format(query=query, context=context)
  return prompt

# Get generate by gemini

In [46]:
import google.generativeai as genai
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GeminiKey')
GEMINI_API_KEY = os.environ['GEMINI_API_KEY']
def generate_by_Gemini(prompt):
  genai.configure(api_key=GEMINI_API_KEY)
  model = genai.GenerativeModel(model_name='gemini-pro')
  answer = model.generate_content(prompt)
  return answer.text

#PDF Processing

In [12]:
pdf_path = '/content/gdrive/MyDrive/AI_integration_kernel/MediBee/AllTF.pdf'
chroma_db_path = '/content/drive/MyDrive/AI_integration_kernel/all_tf_db_3000_500'
model_name ='sentence-transformers/all-MiniLM-L6-v2'

In [13]:
docs =split_pdf(pdf_path, chunk_size=3000, chunk_overlap=500)
create_VDS(docs, chroma_db_path)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of vectors in the vector store: 816


#Generating True False Question

In [18]:
while(True):
  print('---------------------------------------------------------------------------------\n')
  query = input("Please enter topic: ")
  if (query == 'bye'):
    break

  context = get_context_true_false(query, chroma_db_path, model_name, k=12)
  prompt = get_prompt_TrueFalse(query=query,context=context)
  question = generate_by_Gemini(prompt=prompt)
  print(question, '\n\n')

---------------------------------------------------------------------------------

Please enter topic: clavicle 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**Topic: Clavicle**

1. True/False: The clavicle is the only long bone that lies vertically.
2. True/False: The clavicle has a medullary cavity.
3. True/False: The clavicle is the last bone to start ossification.
4. True/False: The acromioclavicular joint is a saddle-type synovial joint.
5. True/False: The sternoclavicular joint is stabilized by the coracoacromial arch.
6. True/False: The pectoralis major muscle flexes the shoulder joint. 


---------------------------------------------------------------------------------

Please enter topic: bye


In [ ]:
while(True):
  print('---------------------------------------------------------------------------------\n')
  query = input("Please enter topic: ")
  if (query == 'bye'):
    break

  context = get_context_sba(query, chroma_db_path, model_name, k=12)
  prompt = get_prompt_SBA(query=query,context=context)
  question = generate_by_Gemini(prompt=prompt)
  print(question, '\n\n')

---------------------------------------------------------------------------------

Please enter topic: Heart sound
**Question 1:**

Which of the following is NOT a characteristic of the first heart sound (S1)?

(A) Low-pitched
(B) Best heard over the mitral region
(C) Indicates closure of the aortic valve
(D) Coincides with the apex beat

**Answer:** (C)

**Question 2:**

Which of the following heart sounds indicates the end of ventricular systole and beginning of ventricular diastole?

(A) First heart sound (S1)
(B) Second heart sound (S2)
(C) Third heart sound (S3)
(D) Fourth heart sound (S4)

**Answer:** (B)

**Question 3:**

Which of the following murmurs is heard during systole?

(A) Aortic regurgitation
(B) Mitral stenosis
(C) Aortic stenosis
(D) Tricuspid regurgitation

**Answer:** (C)

**Question 4:**

An ECG tracing shows an RR interval of 30 mm. What is the heart rate?

(A) 50 beats/min
(B) 75 beats/min
(C) 100 beats/min
(D) 150 beats/min

**Answer:** (A)

**Question 5:**

Wh

In [41]:
def get_answer_prompt(context, question, answer):
  context = context.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""You are assisting a medical student in preparing for an exam. The student has answered a QUESTION and want to check if the ANSWER is correct\
  As an assistant your job is to examine the ANSWER and provide a detailed feedback. You are given the QUESTION and the student's ANSWER to that QUESTION. Also, the relevent CONTEXT from which the QUESTION is asked.\
  Now evaluate the student's answer and give a detailed feedback. If the answer seems incorrect\
  also provide the correct answer. \
  CONTEXT: '{context}'
  QUESTION: '{question}'
  STUDENT'S ANSWER: '{answer}'
  YOUR FEEDBACK: """).format(context=context, question=question, answer=answer)
  return prompt

In [43]:
def get_hint_prompt(query, context, question):
  context = context.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""You are assisting a medical student in preparation for an exam. You are helping student to answer a given QUESTION. Your job is to generate\
  a HINT for the student in order to answer the question. You are given the CONTEXT and the QUESTION from that context. \
  Based on the given CONTEXT you generated the VIVA QUESTION. But I do not know the exact answer. Give me a hint about the answer. Your hint should be \
  small and precise and it should avoid giving out answer. Generate a helpful hint to answer the question. \
  TOPIC: '{query}'
  CONTEXT: '{context}'
  QUESTION: '{question}'
.""").format(query=query, context=context, question=question)
  return prompt



In [47]:
while(True):
  print('---------------------------------------------------------------------------------\n')
  query = input("Please enter topic: ")
  if (query == 'bye'):
    break

  context = get_context_viva(query, chroma_db_path, model_name, k=12)
  prompt = get_prompt_Viva(query=query, context=context)
  question = generate_by_Gemini(prompt=prompt)
  print(question, '\n\n')
  needHint = input("Do you need hint? (y/n): ")
  if(needHint == 'y'):
    prompt = get_hint_prompt(query, context, question)
    hint = generate_by_Gemini(prompt=prompt)
    print('\n', hint, '\n\n')

  answer = input("Provide answer: ")
  prompt = get_answer_prompt(context=context, question=question, answer=answer)
  feedback = generate_by_Gemini(prompt=prompt)
  print('\n\nModel\'s feedback: ', feedback, '\n\n')



---------------------------------------------------------------------------------

Please enter topic: heart sound
**Viva Question:**

What is the third heart sound? 


Do you need hint? (y/n): y

 **Hint:** It occurs in early diastole. 


Provide answer: no idea


Model's feedback:  **Feedback for the Student's Answer:**

The student's answer, "no idea," indicates a lack of knowledge about the third heart sound. Here's a detailed explanation of the third heart sound:

**Definition:**
The third heart sound (S3) is a low-pitched, muffled sound that occurs during the rapid filling phase of the ventricles. It is commonly heard in healthy children and young adults but can become more prominent in older adults or with certain heart conditions.

**Physiological Mechanism:**
The third heart sound is produced by rapid ventricular filling during the early phase of diastole. When the atrioventricular (AV) valves open, blood from the atria rushes into the ventricles, causing the ventricular walls